In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
import pdb
import sys
import pickle
import numpy as np
import scipy as sc
from scipy.io import readsav
from astropy.io import fits
import matplotlib.pyplot as plt

dtor=0.017453292
sky_sqr_deg = 41253 

import warnings
warnings.filterwarnings("ignore")

In [3]:
hipe_8_11_calibration_correction=np.array([1.0253,1.0250,1.0125])
spire_color_correction=np.array([1.0213158,1.0100951,1.0252420])
spire_solid_angle=np.array([459.22,796.98,1649.72])/3600.**2.*(np.pi/180.)**2.

In [4]:
def ell_to_k(ell):
    #ell = 2pi k_theta
    k_theta = ell/2/np.pi * (np.pi/180/60)
    return k_theta

In [5]:
def k_to_ell(k):
    #ell = 2pi k_theta
    ell = k * 2 * np.pi / (np.pi/180/60)
    return ell

In [6]:
def beam_solid_angle_from_fwhm(fwhm):
    sig = (fwhm/3600 * np.pi/180)/2.355
    omega_beam=2*np.pi*sig**2
    return omega_beam

In [7]:
def get_map_native_deltal(mapin, pixsize_arcsec):
    dims = np.shape(mapin)
    eff_side=np.sqrt(np.product(dims))
    eff_a=eff_side*(pixsize_arcsec/3600.)*dtor
    deltal=2.*np.pi/eff_a
    return deltal

In [8]:
name='PSW'
write_maps=False
deltal=120
width=0.95

In [9]:
path_hetdex_maps = os.path.join(os.environ['MAPSPATH'],'cutouts','layers')
layers_directory = 'hetdex_hers_layers_12arcsec_pix_new'
layers_directory = 'hetdex_hers_layers_12arcsec_pix_4x4'
layers_directory = 'hetdex_hers_layers_cea_3x3'
layers_directory = 'hetdex_hers_layers_cea_3x2'
path_hetdex_layers = os.path.join(path_hetdex_maps,layers_directory)

map_bins=np.unique(['__'.join(i.split('__')[:-1]) for i in os.listdir(path_hetdex_layers)])
map_key_dict={'PSW':'__fwhm_17p6','PMW':'__fwhm_23p9','PLW':'__fwhm_35p2'}
map_psf = {'PSW':17.6,'PMW':23.9,'PLW':35.2}

In [10]:
map_bins = [i for i in map_bins if i != ""]

In [ ]:
path_hers_maps = os.path.join(os.environ['MAPSPATH'],'Herschel','HerS')

In [ ]:
hers_maps = {'PSW':'hers-act_SANEPIC_PSW_20121012',
             'PMW':'hers-act_SANEPIC_PMW_20121012',
             'PLW':'hers-act_SANEPIC_PLW_20121012'}

In [ ]:
class CrossMapObject():
    maps={}
    masks={}
    headers={}
    pix_sizes_arcsec={}
    def __init__(self, base_path, jack_dict, crop=False, l20=None, hanning=False, kaiser=False):
        for jack, map_dict in jack_dict.items():
            self.maps[jack]={}
            self.masks[jack]={}
            self.headers[jack]={}
            self.pix_sizes_arcsec[jack]={}
            for key, map_name in map_dict.items():
                map_path = os.path.join(base_path, map_name+'.fits')
                try:
                    map_in, hd_in = fits.getdata(map_path, 0, header=True)
                except:
                    map_in, hd_in = fits.getdata(map_path, 1, header=True)
                try:
                    pix_arcsec = hd_in['CDELT2']*3600
                except:
                    pix_arcsec = hd_in['CD2_2']*3600
                    
                if crop:
                    cx=crop
                    cy=crop
                    dims0 = np.shape(map_in)
                    map_in=map_in[dims0[0]//2-cx//2:dims0[0]//2+cx//2,dims0[1]//2-cy//2:dims0[1]//2+cy//2]
                    
                dims = np.shape(map_in)
                self.headers[jack][key] = hd_in
                self.pix_sizes_arcsec[jack][key] = pix_arcsec
                
                mask_in=np.ones_like(map_in)
                mask_in[np.isnan(map_in)]=0
                # Hanning Mask
                if hanning:
                    window2d = np.sqrt(np.outer(np.abs(np.hanning(dims[0])),np.abs(np.hanning(dims[1]))))
                    mask_in*=window2d
                # Kaiser Mask (beta=kaiser) beta=4 recommended
                if kaiser:
                    window2d = np.sqrt(np.outer(np.abs(np.kaiser(dims[0],kaiser)),np.abs(np.kaiser(dims[1],kaiser))))
                    mask_in*=window2d

                map_in[np.isnan(map_in)]=0
                if l20:
                    self.maps[jack][key] = np.zeros([l20,l20])
                    self.maps[jack][key][:dims[0],:dims[1]] = map_in
                    self.masks[jack][key] = np.zeros([l20,l20])
                    self.masks[jack][key][:dims[0],:dims[1]] = mask_in
                else:
                    self.maps[jack][key] = map_in
                    self.masks[jack][key] = mask_in

In [ ]:
class MapObject():
    maps={}
    masks={}
    headers={}
    pix_sizes_arcsec={}
    def __init__(self, base_path, map_dict, crop=False, l20=None, hanning=False):
        
        for key, map_name in map_dict.items():
            map_path = os.path.join(base_path, map_name+'.fits')
            try:
                map_in, hd_in = fits.getdata(map_path, 0, header=True)
            except:
                map_in, hd_in = fits.getdata(map_path, 1, header=True)
            try:
                pix_arcsec = hd_in['CDELT2']*3600
            except:
                pix_arcsec = hd_in['CD2_2']*3600
            
            if crop:
                cx=crop
                cy=crop
                dims0 = np.shape(map_in)
                map_in=map_in[dims0[0]//2-cx//2:dims0[0]//2+cx//2,dims0[1]//2-cy//2:dims0[1]//2+cy//2]
                
            dims = np.shape(map_in)
            self.headers[key] = hd_in
            self.pix_sizes_arcsec[key] = pix_arcsec
            
            # Hanning Mask
            window2d = np.sqrt(np.outer(np.abs(np.hanning(dims[0])),np.abs(np.hanning(dims[1]))))
            mask_in=np.ones_like(map_in)
            mask_in[np.isnan(map_in)]=0
            if hanning:
                mask_in*=window2d
            map_in[np.isnan(map_in)]=0
            
            if l20:
                self.maps[key] = np.zeros([l20,l20])
                self.maps[key][:dims[0],:dims[1]] = map_in
                self.masks[key] = np.zeros([l20,l20])
                self.masks[key][:dims[0],:dims[1]] = mask_in
            else:
                self.maps[key] = map_in
                self.masks[key] = mask_in

In [ ]:
def get_psf_correction(ell, fwhm_arcsec, fwhm_arcsec2=None):
    sigma_radian1 = (fwhm_arcsec / 3600 * np.pi / 180) / np.sqrt(8 * np.log10(2))
    sigma_ell1 = 1/sigma_radian1
    Bl1 = np.exp(-0.5 *(ell/sigma_ell1)**2)
    if fwhm_arcsec2:
        sigma_radian2 = (fwhm_arcsec2 / 3600 * np.pi / 180) / np.sqrt(8 * np.log10(2))
        sigma_ell2 = 1/sigma_radian2
        Bl2 = np.exp(-0.5 *(ell/sigma_ell2)**2)
        Bl = np.sqrt(Bl1 * Bl2)
        return Bl
    else:
        return Bl1

In [ ]:
def get_twod_fft(map_one, map_two=None, pix_arcsec=False):
    dims = np.shape(map_one)
    if pix_arcsec:
        fac = (pix_arcsec/3600 * (np.pi/180))**2 * (dims[0]*dims[1])
    else:
        fac = 1
        
    if map_two is None:
        return np.real(np.fft.fft2(map_one)*np.conj(np.fft.fft2(map_one))) * fac
    else:
        return np.real(np.fft.fft2(map_one)*np.conj(np.fft.fft2(map_two))) * fac        

In [ ]:
def get_twod_fft(map_one, map_two=None, pix_arcsec=False):
    if map_two is None:
        map_two = map_one.copy()
    dimx, dimy = map_one.shape
    if pix_arcsec:
        sterad_per_pix = (pix_arcsec / 3600 / 180 * np.pi) ** 2
        V = dimx * dimy * sterad_per_pix
    else:
        sterad_per_pix = 1
        V = 1

    ffta = np.fft.fftn(map_one * sterad_per_pix)
    fftb = np.fft.fftn(map_two * sterad_per_pix)
    ps2d = np.real(ffta * np.conj(fftb)) / V
    ps2d = np.fft.ifftshift(ps2d)
    return ps2d    

In [ ]:
def get_twod_ifft(map_one, map_two=None, pix_arcsec=False):
    dims = np.shape(map_one)
    if pix_arcsec:
        fac = (pix_arcsec/3600 * (np.pi/180))**2 * (dims[0]*dims[1])
    else:
        fac = 1
        
    if map_two is None:
        return np.real(np.fft.ifft2(map_one)*np.conj(np.fft.ifft2(map_one))) * fac
    else:
        return np.real(np.fft.ifft2(map_one)*np.conj(np.fft.ifft2(map_two))) * fac        

In [ ]:
def get_twod_ifft(map_one, map_two=None, pix_arcsec=False):
    if map_two is None:
        map_two = map_one.copy()
    dimx, dimy = map_one.shape
    if pix_arcsec:
        sterad_per_pix = (pix_arcsec / 3600 / 180 * np.pi) ** 2
        V = dimx * dimy * sterad_per_pix
    else:
        sterad_per_pix = 1
        V = 1

    ffta = np.fft.fftn(map_one * sterad_per_pix)
    fftb = np.fft.fftn(map_two * sterad_per_pix)
    ps2d = np.real(ffta * np.conj(fftb)) / V
    ps2d = np.fft.ifftshift(ps2d)
    return ps2d    

In [ ]:
def get_k_from_map(mapin, pixsize_arcsec):
    dims=np.shape(mapin)
    return get_kmap_idl(dims, pixsize_arcsec)

In [ ]:
def get_kmap_idl(dims, pixsize_arcsec):

    kx = np.arange(dims[0] // 2 + 1) / ((pixsize_arcsec * dims[0] * np.pi / 10800. / 60.)) * 2. * np.pi
    kx = np.concatenate((kx, -kx[1:((1 + dims[0]) // 2)][::-1]))

    ky = np.arange(dims[1] // 2 + 1) / ((pixsize_arcsec * dims[1] * np.pi / 10800. / 60.)) * 2. * np.pi
    ky = np.concatenate((ky, -ky[1:((1 + dims[1]) // 2)][::-1]))

    lx = len(kx)
    ly = len(ky)
    kx = (np.ones((ly, lx)) * kx)
    ky = (np.ones((lx, ly)) * ky).T
    k = np.sqrt(kx ** 2 + ky ** 2)
    return k.T

In [ ]:
def get_kmap(dims, pixsize_arcsec):
    lx = np.fft.fftfreq(dims[0]) * 2
    ly = np.fft.fftfreq(dims[1]) * 2
    lx = np.fft.ifftshift(lx) * (180 * 3600. / pixsize_arcsec)
    ly = np.fft.ifftshift(ly) * (180 * 3600. / pixsize_arcsec)
    ly, lx = np.meshgrid(ly, lx)
    l2d = np.sqrt(lx ** 2 + ly ** 2)
    return l2d

In [ ]:
def get_ell_bins(mapin, pix_arcsec, deltal=None, width=0, lmin=0, k_theta=False):
    
    if not deltal:
        shape_mapin = np.shape(mapin)
        xx=shape_mapin[0]
        yy=shape_mapin[1]
        eff_side=np.sqrt(xx*yy)
        eff_side= yy
        print('sqrt(xx*yy) = {}, xx={}, yy={}'.format(np.sqrt(xx*yy),xx,yy))
        eff_a=eff_side*(pix_arcsec/3600.)*dtor
        deltal=2.*np.pi/eff_a
        print('deltal is {}'.format(deltal))
        
    if not width:
        dims=np.shape(mapin)
        kmap = get_kmap(dims, pix_arcsec)
        nk = int(np.floor(np.max(kmap)/deltal))
        ell = np.arange(nk)*deltal + deltal/2 + lmin
    else:
        ell = linloggen(deltal,width)
        
    k_theta_log = ell_to_k(ell)
    if k_theta:
        return k_theta_log
    else:
        return ell

In [ ]:
def get_mc_mkk(mask_one, mask_two, pix_arcsec, deltal=None, width=None, nomask=False):
    shape_map = np.shape(mask_one)
    k_map=get_k_from_map(mask_one, pix_arcsec)
    ell_bins = get_ell_bins(mask_one, pix_arcsec, deltal=deltal, width=width)
    npk = len(ell_bins[:-1])
    pk = np.zeros([npk,npk])
    ell = np.zeros_like(pk)
    for iell in range(npk):
        idx_ring = (k_map >= ell_bins[iell]) & (k_map < ell_bins[iell+1])
        idx_not_ring = (k_map < ell_bins[iell]) | (k_map >= ell_bins[iell+1])
        imap_ring = np.ones_like(mask_one)*np.random.normal(size=shape_map)
        imap_ring[idx_not_ring]=0
        
        imode_map = (np.real(np.fft.ifft2(imap_ring))+np.imag(np.fft.ifft2(imap_ring)))
        if nomask:
            imask_mkk = get_twod_fft(imode_map, map_two=None, pix_arcsec=None)
        else:
            #imask_mkk = get_twod_fft(imode_map * mask_one, map_two=imode_map * mask_two, pix_arcsec=pix_arcsec)
            imask_mkk = get_twod_fft(imode_map * mask_one, map_two=imode_map * mask_two, pix_arcsec=None)
        ipk_mask, ipk_ell = bin_in_rings(imask_mkk, ell_bins, k_map)
        pk[iell] = ipk_mask
        ell[iell] = np.mean(k_map[idx_ring])
    return ell_bins, pk

In [ ]:
def shift_twod(seq, x, y):
    return np.roll(np.roll(seq, int(y), axis = 1), int(x), axis = 0)

In [ ]:
def linloggen(deltal, width, minval=None, maxval=None, ltrans=None, verbose=False):

    if not minval:
        minval=deltal/2.
    if not maxval:
        maxval=k_to_ell(4.)
    if verbose:
        print('minval={0:0.1f}'.format(minval))
        print('maxval={0:0.1f}'.format(maxval))
    nlin=int(np.floor(1./width))
    if verbose:
        print('nlin={0:0.1f}'.format(nlin))
    ltrans=nlin*deltal+minval
    if verbose:
        print('ltrans={0:0.1f}'.format(ltrans))
        
    npoints= 2*np.log10(maxval/ltrans)/width
    if verbose:
        print('npoints={0:0.3f}'.format(npoints))

    points = np.arange(np.floor(npoints))/(npoints)
    if verbose:
        print('len(points)={0:0.3f}'.format(len(points)))

    n=int(np.floor(npoints+nlin))
    if verbose:
        print('n={0:0.3f}'.format(n))
        
    ell=np.zeros(n)
    ell[:nlin]=np.arange(nlin)*deltal+minval
    ell[nlin:]=10**( (np.log10(maxval/ltrans))*points + np.log10(ltrans) )

    return ell

In [ ]:
def bin_in_rings(mapin, ell_bins, kmap, kpower=0):
    pk = np.zeros_like(ell_bins[:-1])
    deltal = np.diff(ell_bins)
    ell = ell_bins[:-1] + deltal / 2.
    ind_log = deltal > deltal[0]
    ell[ind_log] = np.sqrt(ell_bins[:-1] * ell_bins[1:])[ind_log]
    for i in range(len(ell_bins[:-1])):
        ind_ell = (kmap >= ell_bins[i]) & (kmap < ell_bins[i + 1])
        pk[i] = np.mean(mapin[ind_ell] * kmap[ind_ell] ** kpower) / \
                ell[i] ** kpower
        #ell[i] = np.mean(kmap[ind_ell])
    return pk, ell